In [2]:
from dotenv import load_dotenv
import os

load_dotenv()
parser_key = os.getenv("llama_parser")#It only parse 1000 pages per day for free or you will have to upgrade the plan 🤑💸
'''https://cloud.llamaindex.ai/api-key'''#and there must be 1200 pages per file max
pinecone_key=os.getenv("pinecone_api_key")
huggingface_key=os.getenv("HUGGINGFACE_TOKEN")
if parser_key:
    print(True)
if pinecone_key:
    print(True)
if huggingface_key:
    print(True)


True
True
True


In [3]:
import nest_asyncio
from llama_parse import LlamaParse


# Apply nest_asyncio to handle asynchronous operations
nest_asyncio.apply()

# Initialize LlamaParse with API key from environment variable
api_key = parser_key  # Ensure your API key is set here or in the environment variables
parser = LlamaParse(
    api_key=api_key,
    result_type="markdown",  # "markdown" and "text" are available
    num_workers=4,  # if multiple files passed, split in `num_workers` API calls
    verbose=True,
)

# Function to handle loading of documents synchronously
def extract_documents_sync(path):
    if os.path.isfile(path):
        return parser.load_data(path)
    elif os.path.isdir(path):
        files = [os.path.join(path, filename) for filename in os.listdir(path)]
        return parser.load_data(files)
    else:
        raise ValueError(f"Path '{path}' is neither a file nor a directory.")

# Function to handle loading of documents asynchronously
async def extract_documents_async(path):
    if os.path.isfile(path):
        return await parser.aload_data(path)
    elif os.path.isdir(path):
        files = [os.path.join(path, filename) for filename in os.listdir(path)]
        return await parser.aload_data(files)
    else:
        raise ValueError(f"Path '{path}' is neither a file nor a directory.")

# Path to the file or directory containing the documents
path = r"C:\Users\Sayan Maity\OneDrive\Desktop\chat_pdf\Research_on_Machine_Learning_and_Its_Algorithms_an.pdf"

# Extract and print the content of the documents synchronously
documents = extract_documents_sync(path)
for doc in documents:
    print(doc.text[:1000])# Print the first 1000 characters of each 




Started parsing the file under job_id 65c110cc-1702-4df0-8c5f-f9e54c6aa4d5
# Journal of Physics: Conference Series

PAPER &#8226; OPEN ACCESS

Research on Machine Learning and Its Algorithms and Development

To cite this article: Wei Jin 2020 J. Phys.: Conf. Ser. 1544 012003

View the article online for updates and enhancements.

IOP ebooks

Bringing together innovative digital publishing with leading authors from the global scientific community.

Start exploring the collection-download the first chapter of every title for free.

This content was downloaded from IP address 158.46.154.149 on 03/06/2020 at 13:35
# Research on Machine Learning and Its Algorithms and Development

Wei Jin

Northwestern Polytechnical University Ming De College, Xi’an, Shaanxi, China

Abstract: This article analyzes the basic classification of machine learning, including supervised learning, unsupervised learning, and reinforcement learning. It combines analysis on common algorithms in machine learning, such 

Embedding with Huggingface

In [5]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
embeddings=HuggingFaceEmbeddings()
db= FAISS.from_documents(documents,embeddings)
# Create prompt template
system_prompt = """Use the given context to answer the question. 
If you don't know the answer, say you don't know. 
Use three sentences maximum and keep the answer concise. 
Context: {context}"""
query="How many machine learning algorithms are there?"
docs=db.similarity_search_with_relevance_scores(query)
print(docs)

OSError: [WinError 126] The specified module could not be found. Error loading "c:\Users\Sayan Maity\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.

pip install python-dotenv llama-parse nest_asyncio langchain langchain_openai faiss-cpu

In [16]:
import os
from dotenv import load_dotenv
from llama_parse import LlamaParse
import nest_asyncio
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.documents import Document

# Load environment variables
load_dotenv()
parser_key = os.getenv("llama_parser")
openai_api_key = os.getenv("openai_api_key")

# Apply nest_asyncio to handle asynchronous operations
nest_asyncio.apply()

# Initialize LlamaParse
parser = LlamaParse(
    api_key=parser_key,
    result_type="markdown",
    num_workers=4,
    verbose=True,
)

# Function to extract documents
def extract_documents_sync(path):
    if os.path.isfile(path):
        return parser.load_data(path)
    elif os.path.isdir(path):
        files = [os.path.join(path, filename) for filename in os.listdir(path)]
        return parser.load_data(files)
    else:
        raise ValueError(f"Path '{path}' is neither a file nor a directory.")

# Path to the file or directory containing the documents
path = r"C:\Users\Sayan Maity\OneDrive\Desktop\chat_pdf\Research_on_Machine_Learning_and_Its_Algorithms_an.pdf"

# Extract documents
documents = extract_documents_sync(path)

# Combine all document texts
combined_text = " ".join([doc.text for doc in documents])

# Create text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)

# Split text into chunks
chunks = text_splitter.split_text(combined_text)

# Create OpenAI embeddings
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

# Create FAISS index and save locally
vectorstore = FAISS.from_texts(chunks, embeddings)
vectorstore.save_local("faiss_index")

# Load the saved index
loaded_vectorstore = FAISS.load_local("faiss_index", embeddings)

# Create retriever
retriever = loaded_vectorstore.as_retriever(search_kwargs={"k": 5})

# Create LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key=openai_api_key)

# Create prompt template
system_prompt = """Use the given context to answer the question. 
If you don't know the answer, say you don't know. 
Use three sentences maximum and keep the answer concise. 
Context: {context}"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
])

# Create question-answer chain
question_answer_chain = create_stuff_documents_chain(llm, prompt)

# Create retrieval chain
chain = create_retrieval_chain(retriever, question_answer_chain)

# Function to answer questions
def answer_question(question):
    result = chain.invoke({"input": question})
    return result["answer"]

# Example usage
if __name__ == "__main__":
    while True:
        question = input("Enter your question (or 'quit' to exit): ")
        if question.lower() == 'quit':
            break
        answer = answer_question(question)
        print(f"\nAnswer: {answer}\n")

Started parsing the file under job_id 05306674-0eca-4ca2-91ee-c83eb26f02b6


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}